In [1]:
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

# Sistemas lineares

Nesta aula exploraremos alguns métodos de resolução de sistemas lineares. Nosso objetivo é conseguir a solução para um sistema de n equações lineares e n incógnitas similar ao sistema abaixo

\begin{eqnarray}
x &+ 2y &+ 3z &= 4 \\
x &+ 3y &+ 3z &= 3 \\
x &+ 2y &+ z &= 0
\end{eqnarray}

A abordagem intuitiva consiste em fazer simplificações a partir de combinações das diferentes linhas do sistema de equações. No caso acima, por exemplo, poderíamos avançar subtraindo a primeira equação menos a terceira ou a segunda menos a primeira. Isto gera as novas equações

\begin{eqnarray}
2z &= 4 \\
y &= -1
\end{eqnarray}

É fácil ver que $z = 2$ e, por substituição, x = 0. Simples, né?

Talvez não. O método intuitivo é eficaz em sistemas de equações pequenos. No então, é comum encontrarmos problemas que envolvam literalmente milhares de variáveis e qualquer tentativa de solução manual pode ser descartada. Precisamos de um método mecânico que possa ser transportado facilmente para um programa de computador.

Esta aula irá apresentar algumas opções.

## Sistemas lineares e matrizes

O primeiro passo para resolver sistematicamente um sistema linear, é escrevê-lo na forma matricial. Isto elimina qualquer menção às variáveis desconhecidas e permite que trabalhemos apenas com números, que é algo que o computador sabe fazer bem:

\begin{equation}
\left[
    \begin{array}{ccc}
    1 & 2 & 3 \\
    1 & 2 & 1 \\
    1 & 3 & 3
    \end{array}
\right]\times
\left[\begin{array}{c}x\\y\\z\end{array}\right]=
\left[\begin{array}{c}4\\3\\0\end{array}\right]
\end{equation}

(Se você não se lembra como faz isto, use a [wikipedia](https://pt.wikipedia.org/wiki/Sistema_de_equa%C3%A7%C3%B5es_lineares)).

## Matrizes em Python

O Numpy possui duas estruturas de dados semelhantes que podem ser utilizadas para representar matrizes. Uma delas é a nossa velha conhecida array:

In [2]:
# Uma matriz é criada a partir de lista de listas
# Alinhamos o como ele estaria disposto em uma matriz
# para facilitar a visualização
M = np.array([[1, 2, 3], 
              [1, 3, 3],
              [1, 2, 1]])

Obtemos a transposta de uma matriz utilizando o atributo `.T` da mesma. Para multiplicar duas matrizes, é necessário utilizar a função `np.dot`. A multiplicação de duas arrays é feita termo a termo e não segue as regras de multiplicação de matrizes em álgebra linear. 

In [3]:
M.T  # transposta

array([[1, 1, 1],
       [2, 3, 2],
       [3, 3, 1]])

In [4]:
np.dot(M, M.T)  # multiplicação de matrizes

array([[14, 16,  8],
       [16, 19, 10],
       [ 8, 10,  6]])

In [5]:
M * M.T  # multiplicação termo a termo

array([[1, 2, 3],
       [2, 9, 6],
       [3, 6, 1]])

Outra opção é utilizar o tipo `np.matrix`. Uma matriz do numpy é muito semelhante a um array, mas se distingue por tratar o símbolo de multiplicação `*` como sendo a multiplicação matricial. É possível criar um objeto do tipo matrix da mesma forma que criamos o array (a partir de uma lista de listas) ou chamando um array já inicializado.

In [6]:
M1 = np.array([[1, 2, 3],
               [1, 3, 3],
               [1, 2, 1]])
M2 = np.matrix(M.T)

Observe que a multiplicação entre M1 e M2 é feita utilizando as regras de multiplicação matricial.

In [7]:
M1 * M2

matrix([[14, 16,  8],
        [16, 19, 10],
        [ 8, 10,  6]])

A função np.dot continua realizando a multiplicação matricial

In [8]:
np.dot(M1, M2)

matrix([[14, 16,  8],
        [16, 19, 10],
        [ 8, 10,  6]])

A escolha entre matrizes e arrays é um tanto pessoal. Fora o comportamento na multiplicação, os dois objetos se comportam de maneiras quase idênticas. Eu particularmente prefiro utilizar arrays porque não acho que a função np.dot seja inconveniente o suficiente para justificar aprender um novo tipo de dados. Arrays e matrizes as vezes se distiguem de formas sutis e prefiro evitar supresas utilizando somente arrays no código inteiro. É assim que seguiremos neste tutorial.

## Escalonamento de matrizes

Escalonamento consiste na realização de combinações lineares entre as linhas de uma matriz para transformá-la numa forma diagonal. Começaremos com o escalonamento para os casos mais simples (já já veremos algumas situações onde o método simples pode ser problemático). 

Nosso objetivo é rearranjar as linhas para que possamos escrever a matriz na forma diagonal superior. Esta forma é fácil de resolver por substituição. Antes vamos nos lembrar da nossa matriz M:

In [9]:
M = M.astype(float); M  # convertemos os números para float para evitar truncamentos errados

array([[ 1.,  2.,  3.],
       [ 1.,  3.,  3.],
       [ 1.,  2.,  1.]])

É necessário também definir o vetor que ocupa o lado direito do nosso sistema linear. Pensamos no sistema linear como sendo $Mu = v$, onde $M$ é uma matriz, $u$ é o vetor solução desconhecido e $v$ é o vetor conhecido no lado direito da equação.

In [10]:
# Mu = v
v = np.array([4., 3., 0.])

O primeiro passo antes de combinar as linhas da matriz é fazer a matriz extendida: anexamos o vetor v como sendo a última coluna da matriz. Desta forma, quando fazemos uma operação do tipo "somar linha 1 com a 2" ela se aplica tanto ao lado esquerdo quanto ao lado direito das equações correspondentes.

O numpy possui a função hstack que "empilha" as matrizes horizontalmente.

In [11]:
np.hstack([M, M])   # mas *não* é isso que queremos :)

array([[ 1.,  2.,  3.,  1.,  2.,  3.],
       [ 1.,  3.,  3.,  1.,  3.,  3.],
       [ 1.,  2.,  1.,  1.,  2.,  1.]])

Se utilizarmos o hstack com [M, v], obtemos um erro: o formato de M (3, 3)  e v (3,) não são compatíveis. Precisamos transformar v em um vetor coluna. Vetor coluna nada mais é que uma matriz n x 1.

In [12]:
v[:, None]  # None cria dimensões extras quando elas são necessárias

array([[ 4.],
       [ 3.],
       [ 0.]])

Então colocamos o resultado na função hstack

In [13]:
Mext = np.hstack([M, v[:, None]]); Mext

array([[ 1.,  2.,  3.,  4.],
       [ 1.,  3.,  3.,  3.],
       [ 1.,  2.,  1.,  0.]])

Agora sim!

O próximo passo é zerar os termos abaixo do elemento Mext[0, 0]

In [14]:
pivo = Mext[0, 0]
Mext[1] = Mext[1] - Mext[1, 0] / pivo * Mext[0]
Mext[2] = Mext[2] - Mext[2, 0] / pivo * Mext[0]
Mext

array([[ 1.,  2.,  3.,  4.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0., -2., -4.]])

A idéia básica das equações acima é subtrair de cada linha a linha do pivo multiplicada pelo termo A/pivo, onde A é o valor que queremos eliminar. Em notação matemática, isto seria 

$$L_{ij} \mapsto L_{ij} - \frac{L_{i0}}{L_{00}} L_0j,$$

onde $L_{0j}$ é a linha do pivo para a primeira iteração.

In [15]:
# Continuaríamos usando M[1, 1] como pivo, mas a matriz já está escalonada!
pivo = Mext[1, 1]
Mext[2] = Mext[2] - Mext[2, 1] / pivo * Mext[1]
Mext

array([[ 1.,  2.,  3.,  4.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0., -2., -4.]])

Traduzimos de volta para um sistema linear e vemos que a resposta é fácil de obter:

\begin{eqnarray}
x &+ 2y &+ 3z &= 4 \\
0 &+ y  &+ 0 &= -1 \\
0 &+ 0  &- 2z &= -4
\end{eqnarray}

Onde encontramos novamente que $z=2$, $y=-1$ e $x=0$.

### Generalizando

Generalizamos o procedimento para uma matriz aleatória de dimensão m x n (onde normalmente, n = m + 1, mas na verdade basta que n > m para o algoritmo funcionar).

In [16]:
m, n = 3, 4
matriz = np.random.uniform(size=(m, n))
matriz

array([[ 0.48198161,  0.98422871,  0.82976294,  0.33444333],
       [ 0.4669344 ,  0.16256605,  0.32366328,  0.41938237],
       [ 0.47518896,  0.45080182,  0.12961868,  0.52464709]])

In [17]:
# Escalonamento
for k in range(m - 1):
    pivo = matriz[k, k]
    for i in range(k + 1, m):
        matriz[i] = matriz[i] - matriz[i, k] / pivo * matriz[k]
matriz

array([[ 0.48198161,  0.98422871,  0.82976294,  0.33444333],
       [ 0.        , -0.79093558, -0.48019491,  0.09538018],
       [ 0.        ,  0.        , -0.37301607,  0.13226304]])

A matriz triangular fica ainda mais simples de trabalhar se o pivô for sempre igual a 1.

In [18]:
# Normalização
for k in range(m):
    matriz[k] = matriz[k] / matriz[k, k]
matriz

array([[ 1.        ,  2.04204622,  1.72156558,  0.69389231],
       [-0.        ,  1.        ,  0.60712266, -0.12059159],
       [-0.        , -0.        ,  1.        , -0.35457733]])

Daí temos imediatamente o valor de z. Podemos calcular o valor das outras coordenadas com o procedimento:

In [19]:
u = np.zeros(m)
u[2] = matriz[2, m]  # este é o valor de z
u[1] = matriz[1, m] - sum(matriz[1, 0:m] * u)  # agora y
u[0] = matriz[0, m] - sum(matriz[0, 0:m] * u)  # e finalmente z

A implementação genérica seria: 

In [20]:
u = np.zeros(m)
for k in reversed(range(m)):
    u[k] = matriz[k, m] - sum(matriz[k, 0:m] * u)
u

array([ 1.11097881,  0.09468034, -0.35457733])

In [21]:
# Conferimos a resposta (quase zero)
np.dot(matriz[:, 0:m], u) - matriz[:, -1]

array([ -1.11022302e-16,   0.00000000e+00,   0.00000000e+00])

## Juntando os pedaços: eliminação de Gauss

Abaixo segue um código que realiza todas as etapas acima de uma vez

In [24]:
def resolver_gauss(M, v):
    """Retorna o vetor solução x para o sistema linear Mx = v"""
    
    m = len(M)
    
    # Cria matriz extendida de floats
    v = np.asarray(v, dtype='float')
    M = np.hstack([M, v[:, None]])
    
    # Escalona matriz
    for k in range(m - 1):
        pivo = M[k, k]
        for i in range(k + 1, m):
            M[i] = M[i] - M[i, k] / pivo * M[k]
            
    # Normaliza pivos
    for k in range(m):
        M[k] = M[k] / M[k, k]
        
    # Resolve sistema triangular superior
    x = np.zeros(m)
    for k in reversed(range(m)):
        x[k] = M[k, m] - sum(M[k, 0:m] * x)
    return x
    

Podemos comparar com a rotina utilizada pelo numpy

In [30]:
M = np.array([[1, 2, 4], 
              [3, 5, 7], 
              [6, 8, 9]])
v = np.array([1, 2, 3])

resolver_gauss(M, v), np.linalg.solve(M, v)

(array([ 0.2,  0. ,  0.2]),
 array([  2.00000000e-01,  -1.11022302e-16,   2.00000000e-01]))

Vemos que as duas soluções são praticamente idênticas. Vamos comparar a performance:

In [33]:
print('Eliminação de Gauss')
%timeit resolver_gauss(M, v)

print('\n\nNumpy')
%timeit np.linalg.solve(M, v)

Eliminação de Gauss
The slowest run took 4.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 51.9 µs per loop


Numpy
The slowest run took 4.15 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 16.6 µs per loop


O numpy é consideravelmente mais rápido pois todo processo é implementado de forma bastante otimizada em FORTRAN.

#### Notas finais

Não cobrimos aqui o importante processo de pivoteamento mencionado em sala de aula. Seguem algumas referências:

* http://www.mat.ufrgs.br/~fabio/linear.pdf
* http://conteudo.icmc.usp.br/pessoas/andretta/ensino/aulas/sme0301-1-11/SistemasLinearesGaussPivoteamento.pdf

# Método de Jacobi

Este é outro método para resolução de sistemas lineares. Trata-se de um método iterativo que produz sucessivas aproximações da solução candidata.

Tudo parte da seguinte fatoração das matrizes:

\begin{eqnarray}
      Mu &= v, \\
(D + R)u &= v,
\end{eqnarray}

onde $D$ é uma submatriz de M com inversa conhecida e $R = M - A$ é o resto. Reorganizando os termos, temos

\begin{eqnarray}
& Du &= v - Ru, \\
& u &= D^{-1}(v - Ru),
\end{eqnarray}

Podemos olhar a equação de baixo e tentar utilizá-la como uma regra de iteração

$$u_{k+1} = D^{-1}(v - Ru_k)$$

A idéia é que se $u_k$ convergir obteremos o valor da solução do sistema linear. No método de Jacobi, escolhemos
a matriz $D$ como sendo a formada apenas pelos elementos da diagonal. Uma matriz diagonal pode ser invertida trivialmente e é possível aplicar a regra acima.


### Exemplo

Definimos matrizes e o vetor v. O sistema linear é Mu = v.

In [43]:
# Matriz
M = np.array([[10, 5, 2],
              [-1, 2, 1],
              [0,  2, 5]])

# Vetor
v = np.array([1, 2, 3])

# Matriz diagonal
D = np.array([[10, 0, 0],
              [ 0, 2, 0],
              [ 0, 0, 5]])
Dinv = np.array([[0.1,   0,   0],
                 [  0, 0.5,   0],
                 [  0,   0, 0.2]])

# Resto
R = M - D

Começamos com um chute inicial para u e repetimos a regra de iteração algumas vezes

In [46]:
u = np.array([1, 1, 1])

for _ in range(10):
    u = np.dot(Dinv, v - np.dot(R, u))
    print(u)

[-0.6  1.   0.2]
[-0.44  0.6   0.2 ]
[-0.24  0.68  0.36]
[-0.312  0.7    0.328]
[-0.3156  0.68    0.32  ]
[-0.304   0.6822  0.328 ]
[-0.3067   0.684    0.32712]
[-0.307424  0.68309   0.3264  ]
[-0.306825  0.683088  0.326764]
[-0.3068968  0.6832055  0.3267648]


Vemos que a resposta converge para um valor determinado.

Agora vamos juntar tudo isso em uma função.

In [48]:
def jacobi(M, v, niter=10):
    """Realiza niter iterações do método de Jacobi."""
    
    # Extraímos a diagonal da matriz como vetor.
    diag = np.diag(M)
    
    # Se chamarmos np.diag num vetor, ela gera uma matriz
    D = np.diag(diag)
    Dinv = np.diag(1 / diag)
    
    # Criamos a matriz resto
    R = M - D
    
    # Realizamos a n iterações do algoritmo
    u = v
    for _ in range(niter):
        u = np.dot(Dinv, v - np.dot(R, u))
    return u

In [49]:
jacobi(M, v)

array([-0.30707197,  0.68323194,  0.3266777 ])

Observe que nem sempre o método de jacobi converge (para falar a verdade, ele diverge para a maioria das matrizes).

In [54]:
M = np.array([[1, 2, 4],
              [3, 5, 7],
              [6, 8, 9]])
print(' Gauss:', resolver_gauss(M, v))
print('Jacobi:', jacobi(M, v))


 Gauss: [ 0.2  0.   0.2]
Jacobi: [ 51189.24210402  23652.34524547  21626.36327457]


Mencionamos o critério das linhas em sala de aula que diz quando uma matriz pode ser resolvida pelo método de Jacobi. (https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Jacobi). O critério das linhas diz que o elemento da diagonal em cada linha da matriz deve ser maior em módulo que a soma dos outros elementos da linha (também em módulo). 